In [1]:
import pandas as pd

# 加载数据集
df = pd.read_csv("car.data", header=None)
# 设置列名
df.columns = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"]
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [12]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1]  # 特征
y = df["class"]  # 目标

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1382, 6), (346, 6), (1382,), (346,))

In [13]:
X_train = pd.get_dummies(X_train).values
X_test = pd.get_dummies(X_test).values
y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1382, 21), (346, 21), (1382, 4), (346, 4))

In [36]:
import tensorflow as tf
import numpy as np


# 模型定义
class Model(object):
    def __init__(self):
        # 根据数据 shape，输入 21 输出 4
        # 只需要保证输入、输出 shape 合适，中间层的节点数可以自行调整
        
        # 随机初始化张量参数
        self.W1 = tf.Variable(tf.random.normal([21, 15]))
        self.b1 = tf.Variable(tf.random.normal([15]))
        self.W2 = tf.Variable(tf.random.normal([15, 4]))
        self.b2 = tf.Variable(tf.random.normal([4]))

    def __call__(self, x):
        # tf.cast 不仅可以转换张量类型，还可以直接将 NumPy 数组转换为相应类型的常量张量
        x = tf.cast(x, tf.float32)  # 转换输入数据类型
        # 线性计算 + RELU 激活
        fc1 = tf.nn.relu(tf.add(tf.matmul(x, self.W1), self.b1))  # 全连接层 1
        fc2 = tf.add(tf.matmul(fc1, self.W2), self.b2)  # 全连接层 2
        return fc2

def loss_fn(model, x, y):
    preds = model(x)
    return tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=y)
    )

def accuracy_fn(logits, labels):
    preds = tf.argmax(logits, axis=1)  # 取值最大的索引，正好对应字符标签
    labels = tf.argmax(labels, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(preds, labels), tf.float32))

In [37]:
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

EPOCHS = 500  # 迭代此时
LEARNING_RATE = 0.02  # 学习率
model = Model()  # 实例化模型类
for epoch in range(EPOCHS):
    with tf.GradientTape() as tape:  # 追踪梯度
        loss = loss_fn(model, X_train, y_train)

    trainable_variables = [model.W1, model.b1, model.W2, model.b2]  # 需优化参数列表
    grads = tape.gradient(loss, trainable_variables)  # 计算梯度

    optimizer = tf.optimizers.Adam(learning_rate=LEARNING_RATE)  # Adam 优化器
    optimizer.apply_gradients(zip(grads, trainable_variables))  # 更新梯度

    accuracy = accuracy_fn(model(X_test), y_test)  # 计算准确度

    # 每 100 个 Epoch 输出各项指标
    if epoch == 0:
        print(f"Epoch [000/{EPOCHS}], Accuracy: [{accuracy:.2f}], Loss: [{loss:.4f}]")
    elif (epoch + 1) % 100 == 0:
        print(
            f"Epoch [{epoch+1}/{EPOCHS}], Accuracy: [{accuracy:.2f}], Loss: [{loss:.4f}]"
        )

InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a float tensor but is a bool tensor [Op:Mul] name: 

In [ ]:
# 答案

In [ ]:
X_train = pd.get_dummies(X_train).values
X_test = pd.get_dummies(X_test).values
y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
import tensorflow as tf

def fully_connected(inputs, weights, biases):
    """
    inputs -- 输入 Variable
    weights -- 权重项 Variable
    biases -- 截距项 Variable
    """
    layer = tf.add(tf.matmul(inputs, weights), biases)  # 输入 x 权重 + 截距
    output = tf.nn.relu(layer)  # RELU 激活
    return output

In [ ]:
x = tf.placeholder(tf.float32, [None, 21])  # 输入特征张量占位符

# 全连接层 1
W1 = tf.Variable(tf.random.uniform([21, 15]))  # 随机初始化权重
b1 = tf.Variable(tf.random.uniform([15]))
fc1 = fully_connected(x, W1, b1)

# 全连接层 2
W2 = tf.Variable(tf.random.uniform([15, 4]))
b2 = tf.Variable(tf.random.uniform([4]))
outs = fully_connected(fc1, W2, b2)

outs  # 输出

In [ ]:
y = tf.placeholder(tf.float32, [None, 4])  # 真实值标签占位符

# 交叉熵损失函数，reduce_mean 的目的是对每个样本的计算结果求平均
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(logits=outs, labels=y))
loss

In [ ]:
train_step = tf.train.AdamOptimizer(0.01).minimize(loss)
train_step

In [ ]:
acc = tf.reduce_mean(tf.cast(tf.math.in_top_k(
    outs, tf.math.argmax(y, 1), k=1), tf.float32))  # 准确率计算

In [ ]:
iters = 1000  # 迭代次数
feed_dict_train = {x: X_train, y: y_train}  # 训练数据
feed_dict_test = {x: X_test, y: y_test}  # 测试数据
init = tf.global_variables_initializer()  # 初始化全局变量
with tf.Session() as sess:
    sess.run(init)
    for i in range(iters):
        if (i+1) % 100 == 0:  # 每间隔 100 次打印 loss 值
            print("Iters [{}/{}], Train Acc [{:.3f}], Test Acc [{:.3f}]".format(
                i+1, iters, acc.eval(feed_dict=feed_dict_train), acc.eval(feed_dict=feed_dict_test)))
        sess.run(train_step, feed_dict=feed_dict_train)